In [1]:
import re
import numpy as np
from scipy import signal

In [2]:
with open('input.txt') as f:
    lines = f.readlines()
lines

['Tile 2647:\n',
 '#....#####\n',
 '.##......#\n',
 '##......##\n',
 '.....#..#.\n',
 '.........#\n',
 '.....#..##\n',
 '#.#....#..\n',
 '#......#.#\n',
 '#....##..#\n',
 '...##.....\n',
 '\n',
 'Tile 1283:\n',
 '######..#.\n',
 '#.#..#.#..\n',
 '..#..#...#\n',
 '.#.##..#..\n',
 '#......#..\n',
 '#.#....##.\n',
 '.#.....#.#\n',
 '#.#..#.#.#\n',
 '.#......##\n',
 '...##.....\n',
 '\n',
 'Tile 3547:\n',
 '#.#.#.###.\n',
 '#.........\n',
 '#....##...\n',
 '#.....#..#\n',
 '#.....#.#.\n',
 '##..##...#\n',
 '#...##....\n',
 '......#..#\n',
 '#...##....\n',
 '.....###.#\n',
 '\n',
 'Tile 1451:\n',
 '##..#.#...\n',
 '#.#.......\n',
 '##.#.....#\n',
 '....#.....\n',
 '...#...##.\n',
 '......#.#.\n',
 '#...##.##.\n',
 '........#.\n',
 '.#.##.#...\n',
 '..##..#...\n',
 '\n',
 'Tile 3137:\n',
 '....#.##.#\n',
 '#....#...#\n',
 '..#.#.....\n',
 '...####..#\n',
 '.#.###...#\n',
 '.......#..\n',
 '##.##.#..#\n',
 '.#.##....#\n',
 '#...#....#\n',
 '..##.##..#\n',
 '\n',
 'Tile 2897:\n',
 '###..#.##.\

In [3]:
tiles = {}
for i in range(0, len(lines), 12):
    tiles[int(re.findall(r'\d+', lines[i])[0])] = np.array([list(l.strip()) for l in lines[i+1:i+11]])

In [4]:
tiles_remaining = list(tiles.keys())
tiles_in_grid = {}
grid = np.zeros((30,30))
tile = tiles_remaining.pop()
grid[15,15] = tile
tiles_in_grid[tile] = (15,15)

def try_fitting(edge, tile, get_edge):
    # Rotations
    for i in range(4):
        if (edge == get_edge(tile)).all():
            return tile
        
        # Flip vertical
        if (edge == get_edge(tile[::-1,:])).all():
            return tile[::-1,:]

        # Flip horizontal
        if (edge == get_edge(tile[:,::-1])).all():
            return tile[:,::-1]
        
        tile = np.rot90(tile)
    
    return None

def try_placing(tile_num_0, tile_num_1):
    tile_0 = tiles[tile_num_0]
    
    # Top
    edge_0 = tile_0[0,:]
    tile_1 = try_fitting(edge_0, tiles[tile_num_1], lambda t: t[-1,:])
    
    if tile_1 is not None:
        return (0, -1), tile_1
    
    # Bottom
    edge_0 = tile_0[-1,:]
    tile_1 = try_fitting(edge_0, tiles[tile_num_1], lambda t: t[0,:])
    
    if tile_1 is not None:
        return (0, 1), tile_1
    
    # Left
    edge_0 = tile_0[:,0]
    tile_1 = try_fitting(edge_0, tiles[tile_num_1], lambda t: t[:,-1])
    
    if tile_1 is not None:
        return (-1, 0), tile_1
    
    # Right
    edge_0 = tile_0[:,-1]
    tile_1 = try_fitting(edge_0, tiles[tile_num_1], lambda t: t[:,0])
    
    if tile_1 is not None:
        return (1, 0), tile_1
    
    return None, None

while tiles_remaining:
    tile_num_1 = tiles_remaining.pop(0)
    
    placed = False
    
    for tile_num_0 in list(tiles_in_grid.keys()):
        offset, tile = try_placing(tile_num_0, tile_num_1)
        
        if offset:
            x, y = tiles_in_grid[tile_num_0]
            x += offset[1]
            y += offset[0]
            tiles_in_grid[tile_num_1] = (x,y)
            grid[x,y] = tile_num_1
            tiles[tile_num_1] = tile
            placed = True
            break
            
    if not placed:
        tiles_remaining.append(tile_num_1)

trimmed = np.apply_along_axis(lambda r: np.trim_zeros(r), axis=1, arr=grid[~np.all(grid == 0, axis=1)])

In [5]:
# Part 1

trimmed[0,0] * trimmed[0,-1] * trimmed[-1,0] * trimmed[-1,-1]

83775126454273.0

In [6]:
# Part 2

image = np.zeros((8*trimmed.shape[0], 8*trimmed.shape[1]))
for i in range(trimmed.shape[0]):
    for j in range(trimmed.shape[1]):
        tile = tiles[trimmed[i,j]][1:-1,1:-1] == '#'
        image[i*8:i*8+8,j*8:j*8+8] = tile

sea_monster = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """
sea_monster = (np.array([list(l) for l in sea_monster.splitlines()]) == '#').astype(int)

# Rotations
matches = []
for i in range(5):
    matches.append((signal.convolve2d(image, sea_monster, mode='valid') == sea_monster.sum()).sum())

    # Flip vertical
    matches.append((signal.convolve2d(image[::-1,:], sea_monster, mode='valid') == sea_monster.sum()).sum())

    # Flip horizontal
    matches.append((signal.convolve2d(image[:,::-1], sea_monster, mode='valid') == sea_monster.sum()).sum())

    image = np.rot90(image)
    
image.sum() - max(matches)*sea_monster.sum()

1993.0